In [1]:

from glob import glob
import re

cc_on_pwc		  full_on_pwc	remover.ipynb
CSRNet			  gram.py	results
data			  __init__.py	results.ipynb
datasets		  labeler	run_spreadsheet.py
DDFlow			  LOI.py	testing.txt
DDFlow_pytorch		  LOI.pyc	thesis-loi-288313-328c2d373141.json
DRNet			  main.py	utils.py
DRNet_pytorch		  multi_run.sh	utils.pyc
finishing_spreadsheet.py  preprocess	v2_compare.ipynb
FlowNetPytorch		  __pycache__	video-to-traffic-flow-information


In [2]:
from datasets import basic_entities

In [3]:
def compile_to_frame_obj(dictio, load_labels=True):
    frame = basic_entities.BasicFrame(dictio['png'])
    if load_labels == True:
        with open(dictio['csv']) as f:
            lines = [line.rstrip() for line in f]
            
        if len(lines) > 1:
            for line in lines[1:]:
                coords = line.split(',')
                frame.add_point((int(coords[0]), int(coords[1])))
    return frame

In [4]:
def load_videos(base_path, load_labels=True):
    png_regex = re.compile('(.*)\/([0-9]+)-([0-9]{10}).([0-9]+).png\Z')
    csv_regex = re.compile('(.*)\/([0-9]+)-([0-9]{10})([0-9]+)-tags.csv\Z')
    frame_dict = {}
    for file in glob('{}*'.format(base_path)):
        png_match = png_regex.match(file)
        csv_match = csv_regex.match(file)
        if csv_match is not None:
            frame_number = int(csv_match.group(2))
            uid = int(csv_match.group(3))
        elif png_match is not None:
            frame_number = int(png_match.group(2))
            uid = int(png_match.group(3))
        else:
            print("Nothing!!! Something went wrong!!")
            print(file)
            break

        if frame_number not in frame_dict:
            frame_dict[frame_number] = {}

        if uid not in frame_dict[frame_number]:
            frame_dict[frame_number][uid] = {}

        if csv_match is not None:
            frame_dict[frame_number][uid]['csv'] = file
        elif png_match is not None:
            frame_dict[frame_number][uid]['png'] = file


    frame_numbers = sorted(list(frame_dict.keys()))
    videos = []
    for begin_frame in frame_dict[0]:
        video = basic_entities.BasicVideo(
            png_regex.match(frame_dict[0][begin_frame]['png']).group(1),
            load_labels
        )

        for frame_num in frame_numbers:
            added = False
            for i in range(-20, 2):
                if begin_frame+frame_num+i in frame_dict[frame_num]:
                    added = True
                    video.add_frame(compile_to_frame_obj(
                        frame_dict[frame_num][begin_frame+frame_num+i],
                        load_labels
                    ))
                    break

            if added == False:
                break
        videos.append(video)
        
    return videos
print(load_videos('data/AmsterdamPeds/'))

[<datasets.basic_entities.BasicVideo object at 0x7feaa85c6970>, <datasets.basic_entities.BasicVideo object at 0x7fea9a108280>, <datasets.basic_entities.BasicVideo object at 0x7fea9830ef40>]
